**IMPORT LIBRARIES**

In [1]:
import cv2
from imutils.video import VideoStream
from imutils.video import FPS
import imutils
import numpy as np
import os
from scipy.spatial import distance

**DEFINE PRIVATE FUNCTIONS**

In [3]:
#Images were too large to view, resized images using this function TO 720 height to fit display and see them
def resize(im,h):
    return imutils.resize(im,height = h)

def set_box(img,x,y,w,h,bold):
    """Will create a red box whose top left coordinate is x (left to right) and y (top to bottom) and width and height w and h respectively"""
    global latest
    im = img.copy()
    rows,cols,dim =  img.shape
    if x<0 or x+w>cols or y<0 or y+h>rows:
        print("set_box - out of bounds")
        latest = -1, -1
        return img
    
    im[y+h:y+h+bold,x:x+w,2] = 255 
    im[y+h:y+h+bold,x:x+w,0] = 0 
    im[y+h:y+h+bold,x:x+w,1] = 0
    
    im[y-bold:y,x:x+w,2] = 255 
    im[y-bold:y,x:x+w,0] = 0 
    im[y-bold:y,x:x+w,1] = 0
    
    im[y-bold:y+h+bold,x-bold:x,2] = 255
    im[y-bold:y+h+bold,x-bold:x,1] = 0
    im[y-bold:y+h+bold,x-bold:x,0] = 0
    
    im[y-bold:y+h+bold,x+w:x+w+bold,2] = 255
    im[y-bold:y+h+bold,x+w:x+w+bold,1] = 0
    im[y-bold:y+h+bold,x+w:x+w+bold,0] = 0;
    
    return im

# Mouse Callback Function - This function will execute and will create Red box where you double-click, and store double click coordinates in variable "latest" (tuple)
def red_box(event,x,y,flags,param):
    global img, ori_img,points,latest,w,h
    if event == cv.EVENT_LBUTTONDBLCLK:
        latest = x,y
        img = ori_img.copy();
        img = set_box(img,x,y,w,h,bold)
        #print("x\t{}\ty\t{}\n".format(x,y))

Prequisites - 
1. You have above libraries installed and imported like above, and you have above functions as well.
2. You provide location to folder where you have frames, manually, like I have provided below
3. You have frames folder inside that location, which containes frames you want to work on and are numbered numerically. (In this ex frame_505.tiff to frame_1000.tiff for ex inside frames folder.)

Specify - 
1. location
2. save name of csv file

***MAIN PROGRAM - FIX COORDINATES***

This program, starts by displaying a frame, you will doule-click on it and it will draw red box at that coordinate. Keep on double-clicking to test and see where you want to generate red box (area you are interested).

1. **Press 't'/'v'** to increase/decrease height of box (h) or 'f'/'g' to increase/decrease width of box (w).
2. **Press 'w'/'s'** to increase/decrease step value.
3. If area inside red box is not clear, **press 'c'** and new window will show region inside red box without resizing. (Will automatically close after 5 sec)
4. Everything looks good for current frame? **press 'n'**, *coordinates* of top left point of red box will be added to points, along with *width* and *height* of box. (Points - frame_no, x,y,w,h)
5. If you want to go back, **press 'b'**, last point appended to variable Points will be removed and you will move back same step value of frames backward and that frame will be displayed.

After you have analysed number of frames you want,**Points** will contain info of frames as frame number, x, y, w, h

In [4]:
#-------------------------------------------------FIX COORDINATES-----------------------------------------------
#---------------------------------------------------------------------------------------------------------------

import numpy as np
import cv2 as cv

#-------------------------------------------SPECIFY MANUALLY---------------------------------------------------------
location = "D:\\Sequence Data Feb\\Seq 5 shelly 020\\10\\"
save_file_name = location + "temp_tracking.csv"
#-------------------------------------------------------------------------------------------------------------------


img_loc = location + "frames\\"                                            #Images/frames location
#-------------------------------------------SPECIFY MANUALLY-----------------------------------------------------------
bold = 5
start_frame_no = int(sorted(os.listdir(img_loc))[0].split('.')[0].split('_')[1]) #Check file directory and infer start frame number
end_frame_no = start_frame_no + 550
H = 1080
W = 1920
#-------------------------------------------------------------------------------------------------------------------

ori_img = cv2.imread(img_loc+"frame_{}.tiff".format(start_frame_no))
ori_dimen = ori_img.shape

ori_img = resize(ori_img,720)
resize_dimen = ori_img.shape

h = round((resize_dimen[0]/ori_dimen[0])*H)
w = round((resize_dimen[1]/ori_dimen[1])*W)

y_ratio = ori_dimen[0]/resize_dimen[0]
x_ratio = ori_dimen[1]/resize_dimen[1]

step = 10
action = []
points = []
out = False
frame_no = start_frame_no
while frame_no < end_frame_no:

    ori_img = cv2.imread(img_loc+"frame_{}.tiff".format(frame_no))
    ori_img = resize(ori_img,720)
    img = ori_img.copy()
    cv.namedWindow('check box')
    cv.setMouseCallback('check box',red_box)
    latest = tuple()

    while(1):
        cv.imshow('check box',img)
        key = cv.waitKey(20) & 0xFF
        if key==ord('q'):
            out = True
            break
        elif key==ord('n'):
            if latest:
                x,y = latest
                x_o = round(x*x_ratio) if x>=0 else -1
                y_o = round(y*y_ratio) if y>=0 else -1
                w_o = round(w*x_ratio) if x>=0 else -1
                h_o = round(h*y_ratio) if x>=0 else -1
                points.append((frame_no,x_o,y_o,w_o,h_o))
                action.append(step)
                print("frame\t{}\tx\t{}\ty\t{}\tw\t{}\th\t{}".format(frame_no,x_o,y_o,w_o,h_o))
                frame_no += step
                if frame_no>end_frame_no:
                    out = True
                    break
                ori_img = cv2.imread(img_loc+"frame_{}.tiff".format(frame_no))
                try:
                    ori_img = resize(ori_img,720)
                except:
                    out = True
                    break
                img = ori_img.copy()
        elif key == ord('b'):
            if len(points)>0 and len(action)>0:
                points.pop()
                frame_no -= action.pop()
                ori_img = cv2.imread(img_loc+"frame_{}.tiff".format(frame_no))
                try:
                    ori_img = resize(ori_img,720)
                except:
                    out = True
                    break
                img = ori_img.copy()
        if key==ord('w'):
            step+=4
            print("Step+=4 ---> {}".format(step))
        elif key == ord('s'):
            if step>=5:
                step-=4
                print("Step-=4 ---> {}".format(step))
        if key==ord('c'):
            x,y = latest
            x_o = round(x*x_ratio) if x>=0 else -1
            y_o = round(y*y_ratio) if y>=0 else -1
            w_o = round(w*x_ratio) if x>=0 else -1
            h_o = round(h*y_ratio) if x>=0 else -1
            
            close_img = cv2.imread(img_loc+"frame_{}.tiff".format(frame_no))
            close_img = close_img[y_o:y_o+h_o, x_o:x_o+w_o,:]
            cv2.imshow("CLOSE LOOK",resize(close_img,400))
            cv2.waitKey(5000)
            cv2.destroyWindow("CLOSE LOOK")
        if key==ord('g'):
            w+=5
        elif key==ord('f'):
            if w-5>0:
                w-=5
        if key==ord('t'):
            h+=5
        elif key ==ord('v'):
            if h-5>0:
                h-=5
            
    if out:
        break
cv.destroyAllWindows()


print("\n\nVALUE OF POINTS -----> ")
points

frame	51472	x	5743	y	942	w	1920	h	1080
Step+=4 ---> 14
Step+=4 ---> 18
Step+=4 ---> 22
Step+=4 ---> 26
Step+=4 ---> 30
frame	51482	x	5725	y	828	w	1920	h	1080
frame	51512	x	4501	y	684	w	1920	h	1080
frame	51542	x	4495	y	684	w	1920	h	1080


VALUE OF POINTS -----> 


[(51472, 5743, 942, 1920, 1080),
 (51482, 5725, 828, 1920, 1080),
 (51512, 4501, 684, 1920, 1080),
 (51542, 4495, 684, 1920, 1080)]

In [5]:
#--------------------WRITE CSV DATA FOR ALL FRAMES (INCLUDING IN BETWEEN - LINEARLY)----------------------

with open(save_file_name,'w') as f:
    point_index = 0
    frame_no = start_frame_no
    f.write("frame_no, x, y, width_of_box, height_of_box\n")
    while frame_no<= start_frame_no+530:
        print(frame_no)
        if point_index+1==len(points):
            break
        elif frame_no>points[point_index+1][0]:
            print("point index ++")
            point_index+=1
        else:
            print("Lets solve")
            if points[point_index][1]==-1:
                x_ans, y_ans, w_ans, h_ans = -1,-1,-1,-1
            elif points[point_index+1][1]==-1:
                pass
            else:
                curr_fr, x, y, w, h = points[point_index]
                next_fr, xn, yn, wn, hn = points[point_index+1]
            
                x_ans = round( x + (frame_no-curr_fr)*(xn-x)/(next_fr-curr_fr) )
                y_ans = round( y + (frame_no-curr_fr)*(yn-y)/(next_fr-curr_fr) )
                w_ans = round( w + (frame_no-curr_fr)*(wn-w)/(next_fr-curr_fr) )
                h_ans = round( h + (frame_no-curr_fr)*(hn-h)/(next_fr-curr_fr) )
                
            print("{}, {}, {}, {}, {}\n".format(frame_no,x_ans,y_ans,w_ans,h_ans))
            f.write("{}, {}, {}, {}, {}\n".format(frame_no,x_ans,y_ans,w_ans,h_ans))
            frame_no += 1



51472
Lets solve
51472, 5743, 942, 1920, 1080

51473
Lets solve
51473, 5741, 931, 1920, 1080

51474
Lets solve
51474, 5739, 919, 1920, 1080

51475
Lets solve
51475, 5738, 908, 1920, 1080

51476
Lets solve
51476, 5736, 896, 1920, 1080

51477
Lets solve
51477, 5734, 885, 1920, 1080

51478
Lets solve
51478, 5732, 874, 1920, 1080

51479
Lets solve
51479, 5730, 862, 1920, 1080

51480
Lets solve
51480, 5729, 851, 1920, 1080

51481
Lets solve
51481, 5727, 839, 1920, 1080

51482
Lets solve
51482, 5725, 828, 1920, 1080

51483
point index ++
51483
Lets solve
51483, 5684, 823, 1920, 1080

51484
Lets solve
51484, 5643, 818, 1920, 1080

51485
Lets solve
51485, 5603, 814, 1920, 1080

51486
Lets solve
51486, 5562, 809, 1920, 1080

51487
Lets solve
51487, 5521, 804, 1920, 1080

51488
Lets solve
51488, 5480, 799, 1920, 1080

51489
Lets solve
51489, 5439, 794, 1920, 1080

51490
Lets solve
51490, 5399, 790, 1920, 1080

51491
Lets solve
51491, 5358, 785, 1920, 1080

51492
Lets solve
51492, 5317, 780, 1920

In [207]:
## ------------------------PREVIEW THE BOX YOUR GENERATED CSV CAPTURED------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------

#location = "D:\\location\\to\\your\\folder"
#save_file_name = location + "name_of_file.csv"
#img_loc = location + "frames\\"

with open(save_file_name,'r') as f:
    start_frame_no = int(sorted(os.listdir(img_loc))[0].split('.')[0].split('_')[1])
    shapey = cv2.imread(img_loc+"frame_{}.tiff".format(start_frame_no)).shape
    line = f.readline()
    line = line.split(',')
    print(line[0], line[1], line[2], line[3], line[4])
    line = f.readline()
    while line:
        line = line.split(',')
        frame_no,x,y,w,h = int(line[0]),int(line[1]),int(line[2]),int(line[3]),int(line[4])
        print(frame_no,x,y,w,h)
        if x>=0 and x+w<shapey[1] and y>=0 and y+h<shapey[0]:
            image = cv2.imread(img_loc+"frame_{}.tiff".format(frame_no))[y:y+h,x:x+w,:]
        cv2.imshow("Tracking shot preview",resize(image,720))
        key = cv.waitKey(1) & 0xFF
        if key==ord('q'):
            break
        line = f.readline()
        
cv.destroyAllWindows()

34038 7526 1524 1920 1080
34039 7477 1521 1920 1080
34040 7428 1518 1920 1080
34041 7378 1515 1920 1080
34042 7329 1512 1920 1080
34043 7280 1509 1920 1080
34044 7231 1506 1920 1080
34045 7182 1503 1920 1080
34046 7132 1500 1920 1080
34047 7083 1497 1920 1080
34048 7034 1494 1920 1080
34049 6990 1495 1920 1080
34050 6945 1496 1920 1080
34051 6901 1498 1920 1080
34052 6856 1499 1920 1080
34053 6812 1500 1920 1080
34054 6768 1501 1920 1080
34055 6723 1502 1920 1080
34056 6679 1504 1920 1080
34057 6634 1505 1920 1080
34058 6590 1506 1920 1080
34059 6547 1507 1920 1080
34060 6505 1507 1920 1080
34061 6462 1508 1920 1080
34062 6420 1508 1920 1080
34063 6377 1509 1920 1080
34064 6334 1510 1920 1080
34065 6292 1510 1920 1080
34066 6249 1511 1920 1080
34067 6207 1511 1920 1080
34068 6164 1512 1920 1080
